# Experiment Logs

Log and number each experiment using dataframes

Can move this to doing "online" each time an experiment is run, eventually

Pulling data takes time, especially for entire USGS. So organize folders as:
* exps1
    * unfiltered
    * exp1-1
        * gage+SWOT
        * gage+SWOT+cal
        * fit+stats
    * exp1-2
    * ...
* exps2
...

where each "expsn" directory shares a common set of SWOT data, and you can run the calibration for each nested experiment expsn-m without repulling.


In [1]:
import pandas as pd
from pathlib import Path

Things to log
* expid: experiment id, equal to folder name
* reachdomain: what reaches this runs over
* swot source

In [2]:
def ExpDataFrame(expsid,expid,reachdomain,swotsource,slopedata,slopeminimum=None,slopeconst=0,areaoption='fd',
                constrainhw=False,flowlaw='MWAPN',darkfracmax=None):
    
    '''
        expsid: experiment set id
        expid: experiment id
        reachdomain: name of the set reaches over which to run - 'connecticut+willamette calval gages' or 'usgs' or...
        swotsource: source of swot data: hydrochron, or ADT-pvd (pseudo-version D data from algorithm team)
        slopedata: data element to use for slope - either slope or slope2
        slopeminimum: applied as a normal threshold filter: throws out data with slope is less than the minimum
        slopeconst: slope consistentizer: when slope is less than a minimum value, apply some consistency algorithm. several options
            0 : Colin's suggestion to set all values of slope equal to SWORD if any value in the timeseries is less than 0 (or some positive number very close to 0)
           -1 : Don't apply this at all (in which case a slopeminimum value needs to be set)
            S : where S is a positive number. if slope is less than S, then set slope to S. slopeminimum shouldn't be set
        areaoption : how to compute cross-sectional area
            'fd' : finite difference - the old MetroMan approach
            'fh' : fluvial hypsometry - the approach in Confluence
        constrainhw: switch to use the constrained height-width estimates
        flowlaw: name of flow law to use, in FLaPE-Byrd
            MWAPN: Manning's equation, wide-river approximation, area formulation, power law on Manning's n vs hydraulic depth
            AHGD: at-a-station hydraulic geometry for river depth - i.e. a rating curve
        darkfracmax : or None to use default value
            
    '''
    
    if not darkfracmax:
        darkfracmax=0.4
    
    df=pd.DataFrame(data={
        'expsid':[expsid],
        'expid':[expid],
        'reachdomain':[reachdomain],
        'swotsource':[swotsource],
        'slopedata':[slopedata],  
        'slopeminimum':[slopeminimum], 
        'slopeconsistencycheck':[slopeconst],
        'areaopt':[areaoption],
        'constrainhw':[constrainhw],
        'flowlaw':[flowlaw],
        'darkfracmax':[darkfracmax]
        }
    )
    
    return df

In [3]:
ExpData=Path('ExperimentData/')

In [4]:
dfs=[]

In [5]:
# experiment 1: run over the original 9 calval gages using Hydrochron data
dfs.append(ExpDataFrame('exps1','exp1-1','connecticut+willamette calval gages','hydrochron','slope'))
dfs.append(ExpDataFrame('exps1','exp1-2','connecticut+willamette calval gages','hydrochron','slope2'))
dfs.append(ExpDataFrame('exps1','exp1-3','connecticut+willamette calval gages','hydrochron','slope',slopeminimum=1.7e-5,slopeconst=-1))
dfs.append(ExpDataFrame('exps1','exp1-4','connecticut+willamette calval gages','hydrochron','slope',slopeminimum=3.4e-5,slopeconst=-1))
dfs.append(ExpDataFrame('exps1','exp1-5','connecticut+willamette calval gages','hydrochron','slope',slopeconst=1.7e-5))
dfs.append(ExpDataFrame('exps1','exp1-6','connecticut+willamette calval gages','hydrochron','slope',areaoption='fh'))
dfs.append(ExpDataFrame('exps1','exp1-7','connecticut+willamette calval gages','hydrochron','slope',areaoption='fh',constrainhw=True))
dfs.append(ExpDataFrame('exps1','exp1-8','connecticut+willamette calval gages','hydrochron','slope',flowlaw='AHGD'))
dfs.append(ExpDataFrame('exps1','exp1-9','connecticut+willamette calval gages','hydrochron','slope',darkfracmax=0.2))


In [6]:
# experiment 2: run over all USGS gages using Hydrochron data
dfs.append(ExpDataFrame('exps2','exp2-1','usgs gages','hydrochron','slope'))
dfs.append(ExpDataFrame('exps2','exp2-3','usgs gages','hydrochron','slope',slopeminimum=1.7e-5,slopeconst=-1))
dfs.append(ExpDataFrame('exps2','exp2-4','usgs gages','hydrochron','slope',slopeminimum=3.4e-5,slopeconst=-1))
dfs.append(ExpDataFrame('exps2','exp2-5','usgs gages','hydrochron','slope',slopeconst=1.7e-5))
dfs.append(ExpDataFrame('exps2','exp2-6','usgs gages','hydrochron','slope',areaoption='fh'))
dfs.append(ExpDataFrame('exps2','exp2-8','usgs gages','hydrochron','slope',flowlaw='AHGD'))
dfs.append(ExpDataFrame('exps2','exp2-9','usgs gages','hydrochron','slope',darkfracmax=0.2))
dfs.append(ExpDataFrame('exps2','exp2-10','usgs gages','hydrochron','slope',flowlaw='AHGW'))
dfs.append(ExpDataFrame('exps2','exp2-11','usgs gages','hydrochron','slope',flowlaw='AHGD',darkfracmax=0.2))



In [7]:
# experiment 3: run over 9 calval gages using pseudo-version D data from ADT
dfs.append(ExpDataFrame('exps3','exp3-1','connecticut+willamette calval gages','ADT-pvd','slope'))

In [8]:
df=pd.concat(dfs,ignore_index=True)
df

,expsid,expid,reachdomain,swotsource,slopedata,slopeminimum,slopeconsistencycheck,areaopt,constrainhw,flowlaw,darkfracmax
0,exps1,exp1-1,connecticut+willamette calval gages,hydrochron,slope,None,0.000000,fd,False,MWAPN,0.4
1,exps1,exp1-2,connecticut+willamette calval gages,hydrochron,slope2,None,0.000000,fd,False,MWAPN,0.4
2,exps1,exp1-3,connecticut+willamette calval gages,hydrochron,slope,0.000017,-1.000000,fd,False,MWAPN,0.4
3,exps1,exp1-4,connecticut+willamette calval gages,hydrochron,slope,0.000034,-1.000000,fd,False,MWAPN,0.4
4,exps1,exp1-5,connecticut+willamette calval gages,hydrochron,slope,None,0.000017,fd,False,MWAPN,0.4
5,exps1,exp1-6,connecticut+willamette calval gages,hydrochron,slope,None,0.000000,fh,False,MWAPN,0.4
6,exps1,exp1-7,connecticut+willamette calval gages,hydrochron,slope,None,0.000000,fh,True,MWAPN,0.4
7,exps1,exp1-8,connecticut+willamette calval gages,hydrochron,slope,None,0.000000,fd,False,AHGD,0.4
8,exps1,exp1-9,connecticut+willamette calval gages,hydrochron,slope,None,0.000000,fd,False,MWAPN,0.2
9,exps2,exp2-1,usgs gages,hydrochron,slope,None,0.000000,fd,False,MWAPN,0.4


In [9]:
# check for duplicate expid
exps=list(df['expid'])
nexps=len(exps)
print('there are ',nexps,' experiments')
nu=len(list(set(exps)))
if nu != nexps:
    print('Oops! There are ',nu,' unique experiment directories and this number does not equal number of experiments')


there are  19  experiments


In [10]:
df.to_csv(ExpData.joinpath('experimentlog.csv'))